In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import math

In [2]:
#os.chdir("c:/Users/prebe/Documents/Ellen/Bachelor/Ellen - bachelor")
os.chdir("/Users/Ellen/Documents/Dokumenter/DTU/7. Semester/Bachelor/CSV files")

poiNames = pd.read_csv("POInames.csv")
dfProb = pd.read_csv("ProbAll.csv")

In [3]:
poiStop = poiNames['StopName'].tolist()
stop = ['home','work','other']

#List with all stops
stop.extend(poiStop)

In [4]:
#probability dataframe to matrix
p = dfProb.to_numpy()

In [5]:
#selected stops
stopSelect = ['home', 'work', 'other', 'School', 'Restaurant', 'University', 'Bar', 'Train Station']

In [6]:
#stopIndex of the 10 most probable stops from home
stopIndex = dfProb.sum(axis=0).nlargest(10).index

In [7]:
#name of the 10 most probable stops from home
largeStop = []

for i in range(0,len(stopIndex)):
    largeStop.append(stop[int(stopIndex[i])])

In [8]:
#merge the two lists with the selected stops and the most probable stops
stopSelect.extend(largeStop)

In [9]:
#unique stops from the selected and most probable stops. 
#this is the stops that will be used to the clustering
StopList = list(set(stopSelect))

In [10]:
#the index of the stops that should be used for the clustering
StopIndex = []
for i in range(0, len(StopList)):
    StopIndex.append(stop.index(StopList[i]))

In [11]:
#probability matrix with all users, and the selected stop types
pSelect = p[:,StopIndex]

In [12]:
#pselect to dataframe and save as csv
pd.DataFrame(pSelect).to_csv('pSelectAll.csv', index = False)

In [13]:
#StopList(the names) to dataframe and save as csv
pd.DataFrame(StopList).rename(columns={0: 'Stop'}).to_csv('StopListAll.csv', index = False)

In [14]:
#set seed
# Gap Statistic for K means
def optimalK(data, nrefs, maxClusters):
    """
    Calculates KMeans optimal K using Gap Statistic 
    Params:
        data: ndarry of shape (n_samples, n_features)
        nrefs: number of sample reference datasets to create
        maxClusters: Maximum number of clusters to test for
    Returns: (gaps, optimalK)
    """
    gaps = np.zeros((len(range(1, maxClusters+1)),))
    resultsdf = pd.DataFrame({'clusterCount':[], 'gap':[], 's_k':[], 'gap-s_k':[]})
    for gap_index, k in enumerate(range(1, maxClusters+1)):

# Holder for reference dispersion results
        refDisps = np.zeros(nrefs)

# For n references, generate random sample and perform kmeans getting resulting dispersion of each loop
        for i in range(nrefs):
            
            # Create new random reference set
            randomReference = np.zeros(np.shape(data))
            for j in range (0,np.shape(data)[1]):
                low = min(data[:,j])
                high = max(data[:,j])
                randomReference[:,j] = np.random.uniform(low, high, size=len(data))        
            
            # Fit to it
            km = KMeans(k)
            km.fit(randomReference)
            
            #refDisp 
            refDisps[i] = km.inertia_
              

# Fit cluster to original data and create dispersion
        km = KMeans(k)
        km.fit(data)
        
        origDisp = km.inertia_
# Calculate gap statistic
        gap = (1/nrefs)*np.sum(np.log(refDisps)) - np.log(origDisp)
        l =(1/nrefs)*np.sum(np.log(refDisps))
        sd_k = ((1/nrefs)*np.sum((np.log(refDisps)-l)**2))**(1/2)
        s_k = sd_k*math.sqrt(1+1/nrefs)
        print(k, " Gap: ", gap, " s_k ", s_k)
        
# Assign this loop's gap statistic to gaps
        gaps[gap_index] = gap
    
        resultsdf = resultsdf.append({'clusterCount':k, 'gap':gap, 's_k':s_k, 'gap-s_k':gap-s_k, 'wss':origDisp}, ignore_index=True)
    return (gaps.argmax() + 1, resultsdf)

In [15]:
score_g, df = optimalK(pSelect, nrefs=20, maxClusters=80)

1  Gap:  2.2826736639383878  s_k  0.0008892802933626516
2  Gap:  2.486367613057281  s_k  0.0008401478152215065
3  Gap:  2.6729046763736166  s_k  0.0009578735019962695
4  Gap:  2.743581018942563  s_k  0.0008368050537743966
5  Gap:  2.7946569544009483  s_k  0.0009053829338503559
6  Gap:  2.8417193052865866  s_k  0.0005342239021583859
7  Gap:  2.880734453681069  s_k  0.0006726206289313435
8  Gap:  2.9084729550736164  s_k  0.0007222163492862555
9  Gap:  2.9431786003155223  s_k  0.0010487309008755563
10  Gap:  2.9725031948076426  s_k  0.001048195165588094
11  Gap:  2.999133222988802  s_k  0.0013130578589738372
12  Gap:  3.0240722865365184  s_k  0.0016825700611424804
13  Gap:  3.0474699177228075  s_k  0.0010431319076596394
14  Gap:  3.0684982798752944  s_k  0.001157816667712867
15  Gap:  3.086037239920553  s_k  0.0009312969944849591
16  Gap:  3.098172698769461  s_k  0.0019728215490728424
17  Gap:  3.112027185764134  s_k  0.001053950961062558
18  Gap:  3.127778233291984  s_k  0.00084005546991

In [16]:
np.savetxt('gapSelectAll.csv', df, delimiter=",")

In [17]:
#find the number of clusters, where gap(k) >= gap(k+1)-s_k
#gap1
cluster = []
for i in range(df.shape[0]-1):
    if df['gap'][i] >= df['gap-s_k'][i+1]:
        cluster.append(i+1)
        
cluster

[56, 60, 61, 67, 72, 79]